<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

## `pandas` Long Format, Wide Format, Pivot Tables, and Melting

_Authors: Kiefer Katovich (SF)_

---

This lesson is all about transforming data using `pandas`. Data transformation is the reorganization of your data set's rows and columns into a different, potentially more useful shape and format. 

The benefits of transforming your data include better access to relevant information and streamlined data manipulation. As you become more familiar with data sets and their associated operations, you will develop an intuition and appreciation for when it's better to work row-wise or column-wise.

Different data formats are better for different tasks. It takes time and experience to learn the distinctions. But, for now, we'll introduce the common structures, transformations, and how to apply these transformations.

### Learning Objectives
- Understand the differences between long and wide format data.
- Understand pivot tables.
- Practice transforming data between long and wide formats.
- Practice creating pivot tables.
- Learn how to avoid common pitfalls and obstacles in data transformation with `pandas`.


### Lesson Guide

- [Wide Format Data](#wide_format)
- [Load and Examine the NPAS Data](#load_nerdy)
- [Long Format Data](#long_format)
- [Using `pandas`' `.pivot_table()` Function: Long to Wide Format](#pivot_tables)
- [MultiIndex/Hierarchical Indices in `pandas`](#multiindex)
- [Using `pandas`' `.melt()` Function: Wide to Long Format](#melt)
- [Summarizing Data With `.pivot_table()` and Aggregate  Functions](#pivot_table_summarizing)
- [The Inner Workings of the MultiIndex](#examining_multiindex)
- [Getting Rid of the MultiIndex: "Flattening" Data](#multiindex_to_flat)
- [A Preface: Merging and Joining With Long and Wide Format Data](#merging_joining_preface)
- [`pandas`' `.merge()` function: Joining Long Format vs. Wide Format Data](#pandas_merge)


In [1]:
import numpy as np
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

sns.set_style('darkgrid')

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

<a id='wide_format'></a>

### Wide Format Data

---

Between "wide" and "long," wide format data is the more intuitive. It's also a common format for `.csv` files. You've already viewed multiple data sets in wide format throughout this course.

Wide format data is structured so that:

- Unique IDs, subjects, observations, etc. are represented as rows.
- Distinct information categories (variables) are represented as columns. In other words, there is a column for every "variable" with its own unique values.
- This format can often be a more compact matrix, particularly if little or no information is missing.
- It is not as useful for SQL-style operations: It can make it much harder or even impossible to join tables together on a value.
- It can be useful in `pandas` when you need to perform operations on variables **across columns**; for example, multiplying columns together to create a new column.
- It is the data format required for statistical modeling (with few exceptions).

<a id='load_nerdy'></a>

### Load and Examine the "Nerdy Personality Attributes" Data Set

---

This is a pre-cleaned and modified version of the full "Nerdy Personality Attributes" survey, which asked subjects to rate themselves based on questions related to "nerdiness" as well as more general personality traits such as openness and extraversion. Researches also collected demographic information from the subjects.

You can find the raw data [here](http://personality-testing.info/_rawdata/), along with many other sociological surveys.

In this modified version, for the sake of our example, some of the subjects provided data for the survey but not the demographic variables. Because there are missing values and the data is "messy," we have a data cleaning problem.

**Load the data (which is in wide format).** 

In [2]:
nerdy_wide_f = '../datasets/NPAS_parsed_trunc_wide_missing.csv'

nerdy_wide = pd.read_csv(nerdy_wide_f)
print nerdy_wide.shape

(1391, 57)


This data set is in a familiar format in which each column is a variable and each row contains an observation for that variable, corresponding to a distinct subject.

*Wide format implies that all of the information for one distinct subject will be represented in the columns corresponding to that row. A single subject should not be represented in multiple rows of data.*

In [3]:
nerdy_wide.head(3)

,subject_id,academic_over_social,age,anxious,bookish,books_over_parties,calm,collect_books,conventional,critical,...,religion,reserved,socially_awkward,strange_person,sympathetic,urban,voted,was_odd_child,watch_science_shows,writing_novel
0,0,5.0,NaN,1.0,5.0,5.0,7.0,5.0,1.0,1.0,...,NaN,7.0,5.0,5.0,7.0,NaN,NaN,5.0,5.0,3.0
1,1,2.0,50.0,4.0,4.0,4.0,6.0,5.0,1.0,3.0,...,1.0,5.0,5.0,4.0,5.0,2.0,1.0,3.0,5.0,1.0
2,2,5.0,22.0,7.0,5.0,5.0,2.0,5.0,1.0,6.0,...,1.0,7.0,5.0,5.0,2.0,1.0,1.0,5.0,5.0,4.0


**Check to see how many null values there are per column.**

In [4]:
nerdy_wide.isnull().sum()

subject_id                        0
academic_over_social              0
age                             691
anxious                           0
bookish                           0
books_over_parties                0
calm                              0
collect_books                     0
conventional                      0
critical                          0
dependable                        0
diagnosed_autistic                0
disorganized                      0
education                       691
engnat                          691
enjoy_learning                    0
excited_about_research            0
extraverted                       0
familysize                      691
gender                          691
hand                            691
hobbies_over_people               0
in_advanced_classes               0
intelligence_over_appearance      0
interested_science                0
introspective                     0
libraries_over_publicspace        0
like_dry_topics             

The 691 missing demographic variables are intentional (I specifically enforced that only 700 of the subjects would have demographic information).

However, we can see that the `major` variable has 970 missing values. This was not an intentional change.

At this point, if we were to just drop all the rows that have any null values, we would lose at least 970 rows because of the missing `major` variable.

With a numeric column, this would be hard to avoid without "imputing" some number to fill in those values. In the simplest case, imputing the mean or median for missing numeric values is a common fix (but not ideal).

With a **categorical variable** like `major`, we have the luxury of replacing the missing values with a new category label that stands for "missing." 

**Replace the missing `major` column values with `unknown`.**

In [5]:
nerdy_wide.loc[nerdy_wide.major.isnull(), 'major'] = 'unknown'
print nerdy_wide.major.isnull().sum()

0


<a id='long_format'></a>

### Long Format Data

---

Now, we can load the same data — this time in the format commonly called "long."

Long format data is structured so that:

- There are potentially multiple `ID` (identification) columns.
- There are pairs of columns such as `variable:value` that match a variable key to a value (In the simplest case, there would be a single `variable` column and a single `value` column).
- The `variable` column corresponds to the multiple variable columns in a wide format data set. Instead of a column for each variable, you have a row for each `variable:value` pair *per ID*. 
- This is a standard format for SQL databases because it makes it easier to join different tables together with keys.

**Load the long format of the same data below.**

In [6]:
nerdy_long_f = '../datasets/NPAS_parsed_trunc_long_missing.csv'

nerdy_long = pd.read_csv(nerdy_long_f)
print nerdy_long.shape

(70295, 3)


You can see that the long format data has far more rows than the wide data set but only three columns.

Below you can view the three columns: `subject_id`, `variable`, and `value`.

**`subject_id:`**
- This is the primary "key" or `ID` column. Each `subject_id` will have corresponding entries in the `variable` column — one for each row.

**`variable:`**
- This column indicates the variable with which the item in the `value` column corresponds.

**`value:`**

- This contains all values for all variables for all IDs. Essentially, every cell in the wide data set except the `subject_id` is listed in this column.

In [7]:
nerdy_long.head()

,subject_id,variable,value
0,1,education,4.0
1,2,education,3.0
2,5,education,2.0
3,6,education,2.0
4,7,education,2.0


**Print out the unique values in the `variable` column.**

You can see that the unique values in the `variable` column correspond to the column headers in the wide format data.

In [8]:
nerdy_long.variable.unique()

array(['education', 'urban', 'gender', 'engnat', 'age', 'hand', 'religion',
       'voted', 'married', 'familysize', 'major', 'race_white',
       'race_nerdy', 'race_native_american', 'writing_novel',
       'read_tech_reports', 'online_over_inperson', 'introspective',
       'hobbies_over_people', 'books_over_parties', 'bookish',
       'libraries_over_publicspace', 'race_native_austrailian',
       'like_hard_material', 'race_hispanic', 'diagnosed_autistic',
       'play_many_videogames', 'race_arab', 'race_asian',
       'interested_science', 'playes_rpgs', 'in_advanced_classes',
       'collect_books', 'intelligence_over_appearance',
       'watch_science_shows', 'academic_over_social',
       'like_science_fiction', 'like_dry_topics', 'race_black', 'calm',
       'disorganized', 'extraverted', 'dependable', 'critical',
       'opennness', 'anxious', 'sympathetic', 'reserved', 'conventional',
       'was_odd_child', 'prefer_fictional_people', 'enjoy_learning',
       'excited_abou

In [9]:
len(nerdy_long.subject_id.unique())

1391

**Replace the missing values in `major` with `unknown` in the long format data set.**

The process for replacing data will be different because of the format. Using logical selection masks with `pandas`' `.loc` syntax is the preferable way to do this.

In [10]:
# Rather than calling the `major` column like we would in a wide DF,
# we have to isolate all rows with a variable value of `major` in a long DF.
major_mask = (nerdy_long.variable == 'major') & (nerdy_long.value.isnull())
nerdy_long.loc[major_mask, 'value'] = 'unknown'
print nerdy_long[nerdy_long.variable == 'major'].isnull().sum()

subject_id    0
variable      0
value         0
dtype: int64


<a id='pivot_tables'></a>

### `Pandas`' `.pivot_table()` Function: Long to Wide Format

---

The `pd.pivot_table()` function is a powerful tool for both transforming data from long to wide format as well as summarizing data with user-supplied functions.

First, we'll look at transforming the long format data back into the wide format using the `.pivot_table()` function.

**Important parameters for the `.pivot_table()` function include:**

> The `pivot_table()` function takes a DataFrame to pivot as its first argument. 
    
- **`columns`**: This is the list of columns in the long format data to be transformed back into columns in the wide format. After pivoting, each unique value in the long format column becomes a header in the wide format.
- **`values`**: A single column indicating the values to use when pivoting and filling the new wide format columns.
- **`index`**: Columns in the long format data that are index variables. These will be left as single columns, not spread out by unique value like in the `columns` parameter.
- **`aggfunc`**: Often `.pivot_table()` is used to perform a summary of the data. `aggfunc` stands for "aggregation function." It's required and defaults to `np.mean()`. You can also insert your own function, which we'll demonstrate below.
- **`fill_value`**: If a cell is missing for the wide format data, this value will fill it in.
    
Next we'll put in our own function — `select_item_or_nan()` — to the `aggfunc` keyword argument. Because my `subject_id` column has a single variable value for each ID, I just want the single element in the long format value cell. My data is messy, so I have to write a function to check for places it could break. 

**Note:** Passed into my function, `x` will be a Series object. I pull out the first element of that using the `.iloc` indexer.

In [45]:
nerdy_long.head()

,subject_id,variable,value
0,1,education,4.0
1,2,education,3.0
2,5,education,2.0
3,6,education,2.0
4,7,education,2.0


In [48]:
def select_item_or_nan(x):
    x = x.iloc[0]
    if len(x) == 0:
        return np.nan
    else:
        return x
# This will take a few seconds to run.
nerdy_wide_pv = pd.pivot_table(nerdy_long, columns=['variable'], values='value',
                            index=['subject_id'], aggfunc=select_item_or_nan,
                            fill_value=np.nan)
# 'pv' for 'pivot version.'
nerdy_wide_pv.head()

variable,academic_over_social,age,anxious,bookish,books_over_parties,calm,collect_books,conventional,critical,dependable,...,religion,reserved,socially_awkward,strange_person,sympathetic,urban,voted,was_odd_child,watch_science_shows,writing_novel
subject_id,,,,,,,,,,,,,,,,,,,,,
0,5.0,NaN,1.0,5.0,5.0,7.0,5.0,1.0,1.0,7.0,...,NaN,7.0,5.0,5.0,7.0,NaN,NaN,5.0,5.0,3.0
1,2.0,50.0,4.0,4.0,4.0,6.0,5.0,1.0,3.0,5.0,...,1.0,5.0,5.0,4.0,5.0,2.0,1.0,3.0,5.0,1.0
2,5.0,22.0,7.0,5.0,5.0,2.0,5.0,1.0,6.0,3.0,...,1.0,7.0,5.0,5.0,2.0,1.0,1.0,5.0,5.0,4.0
3,5.0,NaN,4.0,4.0,5.0,7.0,5.0,1.0,2.0,7.0,...,NaN,2.0,5.0,5.0,6.0,NaN,NaN,5.0,5.0,4.0
4,4.0,NaN,3.0,5.0,5.0,6.0,4.0,2.0,5.0,4.0,...,NaN,6.0,0.0,5.0,5.0,NaN,NaN,5.0,4.0,1.0


<a id='multiindex'></a>

### MultiIndex/Hierarchical Indices in `pandas`

---

In the header, you can see that the format of the new wide data is *not* the same as our originally loaded wide format. `pandas` implements something called **MultiIndexing** or **hierarchical indexing**, which allows for "tiered" row and column labels.

Right now the mMltiIndex is not terrible but can get confusing and annoying, which we will experience later in this lesson.

The main difference is that we now have a `variable` name in the top left corner, which is "labeling" our columns (and corresponds to the name of our original column in the long format data). The row indexer has become our single key/ID variable, `subject_id`. The columns are what we would expect here: Each one is a variable, like in the original wide format data.

In [12]:
nerdy_wide_pv.head()

variable,academic_over_social,age,anxious,bookish,books_over_parties,calm,collect_books,conventional,critical,dependable,...,religion,reserved,socially_awkward,strange_person,sympathetic,urban,voted,was_odd_child,watch_science_shows,writing_novel
subject_id,,,,,,,,,,,,,,,,,,,,,
0,5.0,NaN,1.0,5.0,5.0,7.0,5.0,1.0,1.0,7.0,...,NaN,7.0,5.0,5.0,7.0,NaN,NaN,5.0,5.0,3.0
1,2.0,50.0,4.0,4.0,4.0,6.0,5.0,1.0,3.0,5.0,...,1.0,5.0,5.0,4.0,5.0,2.0,1.0,3.0,5.0,1.0
2,5.0,22.0,7.0,5.0,5.0,2.0,5.0,1.0,6.0,3.0,...,1.0,7.0,5.0,5.0,2.0,1.0,1.0,5.0,5.0,4.0
3,5.0,NaN,4.0,4.0,5.0,7.0,5.0,1.0,2.0,7.0,...,NaN,2.0,5.0,5.0,6.0,NaN,NaN,5.0,5.0,4.0
4,4.0,NaN,3.0,5.0,5.0,6.0,4.0,2.0,5.0,4.0,...,NaN,6.0,0.0,5.0,5.0,NaN,NaN,5.0,4.0,1.0


**Drop the null values from our recreated wide format data. How many unique subjects do we have?**

Remember our `subject_id` is now the **index**, so we can access it using the `.index` attribute.

In [13]:
nerdy_wide_pv.dropna(inplace=True)
print nerdy_wide_pv.shape
print len(nerdy_wide_pv.index.unique())

nerdy_wide_pv.head()

(700, 56)
700


variable,academic_over_social,age,anxious,bookish,books_over_parties,calm,collect_books,conventional,critical,dependable,...,religion,reserved,socially_awkward,strange_person,sympathetic,urban,voted,was_odd_child,watch_science_shows,writing_novel
subject_id,,,,,,,,,,,,,,,,,,,,,
1,2.0,50.0,4.0,4.0,4.0,6.0,5.0,1.0,3.0,5.0,...,1.0,5.0,5.0,4.0,5.0,2.0,1.0,3.0,5.0,1.0
2,5.0,22.0,7.0,5.0,5.0,2.0,5.0,1.0,6.0,3.0,...,1.0,7.0,5.0,5.0,2.0,1.0,1.0,5.0,5.0,4.0
5,4.0,18.0,5.0,3.0,4.0,4.0,4.0,4.0,3.0,5.0,...,1.0,5.0,4.0,5.0,4.0,3.0,2.0,4.0,5.0,3.0
6,4.0,18.0,1.0,4.0,5.0,6.0,5.0,1.0,1.0,2.0,...,1.0,5.0,5.0,5.0,5.0,2.0,2.0,1.0,4.0,1.0
7,3.0,21.0,7.0,3.0,5.0,1.0,5.0,4.0,6.0,5.0,...,12.0,5.0,5.0,5.0,6.0,2.0,1.0,3.0,3.0,3.0


**Convert the `subject_id` index back into a column.**

We can use the DataFrame function `.reset_index()` to move `subject_id` into a column and create a new index. We now have a DataFrame with the same format we loaded the original wide format data in previously. The only exception is that we still have the `variable` column label.

In [14]:
nerdy_wide_flat = nerdy_wide_pv.reset_index()

In [15]:
nerdy_wide_flat.head(2)

variable,subject_id,academic_over_social,age,anxious,bookish,books_over_parties,calm,collect_books,conventional,critical,...,religion,reserved,socially_awkward,strange_person,sympathetic,urban,voted,was_odd_child,watch_science_shows,writing_novel
0,1,2.0,50.0,4.0,4.0,4.0,6.0,5.0,1.0,3.0,...,1.0,5.0,5.0,4.0,5.0,2.0,1.0,3.0,5.0,1.0
1,2,5.0,22.0,7.0,5.0,5.0,2.0,5.0,1.0,6.0,...,1.0,7.0,5.0,5.0,2.0,1.0,1.0,5.0,5.0,4.0


In [16]:
nerdy_wide_flat.columns.name

'variable'

**Remove the column label.**

You can remove the column label (which can be confusing during print statements) by setting the `.columns.name` attribute to `None`.

In [17]:
nerdy_wide_flat.columns.name = None
nerdy_wide_flat.head(2)

,subject_id,academic_over_social,age,anxious,bookish,books_over_parties,calm,collect_books,conventional,critical,...,religion,reserved,socially_awkward,strange_person,sympathetic,urban,voted,was_odd_child,watch_science_shows,writing_novel
0,1,2.0,50.0,4.0,4.0,4.0,6.0,5.0,1.0,3.0,...,1.0,5.0,5.0,4.0,5.0,2.0,1.0,3.0,5.0,1.0
1,2,5.0,22.0,7.0,5.0,5.0,2.0,5.0,1.0,6.0,...,1.0,7.0,5.0,5.0,2.0,1.0,1.0,5.0,5.0,4.0


<a id='melt'></a>

### Using pandas' `.melt()` Function: Wide to Long Format

---

**`.melt()`** is a function that essentially performs the inverse of `.pivot_table()` on DataFrames.

`.melt()` takes a DataFrame as its first argument. Additional arguments typically used with this function are:

- **`id_vars`**: The column or columns that will be ID variables. ID variables contain data points specified by the `variable` and `value` columns.
- **`value_vars`**: A list that specifies which columns should be converted into single `value` and `variable` columns.
- **`var_name`**: The header name of the `variable` column (default='variable').
- **`value_name`**: The header name of the `value` column (default='value').

**First, subset the wide format data into just columns: `['subject_id','anxious','booking','calm','major']`.**


In [18]:
nerdy_subset = nerdy_wide_flat[['subject_id','major','anxious','bookish','calm']]
nerdy_subset.head(2)

,subject_id,major,anxious,bookish,calm
0,1,biophysics,4.0,4.0,6.0
1,2,biology,7.0,5.0,2.0


**Use `.melt()` on the subset with `id_vars=['subject_id','major']`.**

Print out the shape of the data and the header. The non-ID columns and their values are now represented by the `variable:value` column pair.

**Note**: When you only specify the `id_vars`, the remaining columns become part of the `variable` and `value` columns.

In [19]:
nerdy_sub_long = pd.melt(nerdy_subset, id_vars=['subject_id','major'])
print nerdy_subset.shape, nerdy_sub_long.shape
nerdy_sub_long.head(4)

(700, 5) (2100, 4)


,subject_id,major,variable,value
0,1,biophysics,anxious,4.0
1,2,biology,anxious,7.0
2,5,Geology,anxious,5.0
3,6,unknown,anxious,1.0


If we don't specify `major` as an `id_var`, it will end up in the `variable` column.

In [20]:
nerdy_sub_long = pd.melt(nerdy_subset, id_vars='subject_id')
print nerdy_subset.shape, nerdy_sub_long.shape
nerdy_sub_long.head(4)

(700, 5) (2800, 3)


,subject_id,variable,value
0,1,major,biophysics
1,2,major,biology
2,5,major,Geology
3,6,major,unknown


The more `id_vars` that we specify, the flatter our DataFrame will be. 

You can achieve the same result without having to subset the DataFrame first by simply specifying the `value_vars` keyword argument. The output DataFrame will then only contain the data specified in the `id_vars` and `value_vars` arguments.

**Create the same DataFrame with `.melt()` on the full wide data set, but select the columns to use with the `value_vars` argument.**

In [21]:
# With two `value_vars`:
nerdy_sub_long = pd.melt(nerdy_wide_flat, id_vars=['subject_id','major'], 
                         value_vars=['bookish','calm'])
print nerdy_wide_flat.shape, nerdy_sub_long.shape
nerdy_sub_long.head(4)

(700, 57) (1400, 4)


,subject_id,major,variable,value
0,1,biophysics,bookish,4.0
1,2,biology,bookish,5.0
2,5,Geology,bookish,3.0
3,6,unknown,bookish,4.0


In [22]:
# With all `value_vars`:
nerdy_sub_long = pd.melt(nerdy_wide_flat, id_vars=['subject_id','major'], 
                         value_vars=['anxious','bookish','calm'])
print nerdy_wide_flat.shape, nerdy_sub_long.shape
nerdy_sub_long.head(4)

(700, 57) (2100, 4)


,subject_id,major,variable,value
0,1,biophysics,anxious,4.0
1,2,biology,anxious,7.0
2,5,Geology,anxious,5.0
3,6,unknown,anxious,1.0


In [23]:
nerdy_sub_long.dtypes

subject_id     int64
major         object
variable      object
value         object
dtype: object

The `value` column is still a string, so we can convert it to a float.

In [24]:
nerdy_sub_long.value = nerdy_sub_long.value.astype(float)

<a id='pivot_table_summarizing'></a>

### Summarizing Your Data With  `.pivot_table()` and Aggregate Functions

---

For those of you who have experience with Excel, `pandas`' `.pivot_table()` accomplishes the same thing. It's more powerful but harder to use than the spreadsheet version.

`.pivot_table()` can take in a variable, value, and index to group by and apply aggregate functions to summarize the data. 

**Note**: Be careful that your index variable is not pulling out unique rows (For example, `subject_id` by variable would only have one value to send into the aggregate functions).

Below, I am calling the `.pivot_table()` function with:

- The long format data as the first argument.
- `variable` specified as the columns that indicate the variable names (groups).
- `value` specified as the column that contains the data per variable.
- `major` as the index; the rows will be grouped by `major`.
- `np.mean`, `np.median`, `np.std`, and `len` as aggregate functions. These will be calculated for each `major-by-variable` group.
- A `fill_value` of `np.nan` for cells in the output table that have no data.



In [25]:
nerdy_major_summary = pd.pivot_table(nerdy_sub_long, columns=['variable'], values='value',
                                     index=['major'], aggfunc=[np.mean, np.median, np.std, len],
                                     fill_value=np.nan)

The output DataFrame gives you a "hierarchical" column index — the three variables for each aggregate function. The row index is the `major` groups.

If you apply more index variables, the row indices will also become hierarchical! However, this can quickly make for a bloated DataFrame.

In [26]:
nerdy_major_summary.head(10)

mean            \
variable                                             anxious   bookish   
major                                                                    
 None yet                                           7.000000  3.000000   
+ACI-+ACIAIg-hotel and restaurant management+AC...  2.000000  2.000000   
Aerospace Engineer                                  2.000000  2.000000   
Aerospace Engineering                               3.000000  4.000000   
Agricultural Economics                              2.000000  2.000000   
Anthropology                                        5.333333  3.666667   
Anthropology                                        5.000000  4.000000   
Architecture                                        3.000000  4.000000   
Architecture                                        5.000000  1.000000   
Art                                                 4.000000  4.333333   

                                                              median          \
variable                                                calm anxious bookish   
major                                                                          
 None yet                                           3.000000     7.0     3.0   
+ACI-+ACIAIg-hotel and restaurant management+AC...  7.000000     2.0     2.0   
Aerospace Engineer                                  7.000000     2.0     2.0   
Aerospace Engineering                               3.000000     3.0     4.0   
Agricultural Economics                              6.000000     2.0     2.0   
Anthropology                                        4.333333     5.0     4.0   
Anthropology                                        3.000000     5.0     4.0   
Architecture                                        5.666667     4.0     4.0   
Architecture                                        5.000000     5.0     1.0   
Art                                                 5.333333     5.0     4.5   

                                                              std            \
variable                                           calm   anxious   bookish   
major                                                                         
 None yet                                           3.0       NaN       NaN   
+ACI-+ACIAIg-hotel and restaurant management+AC...  7.0       NaN       NaN   
Aerospace Engineer                                  7.0       NaN       NaN   
Aerospace Engineering                               3.0       NaN       NaN   
Agricultural Economics                              6.0       NaN       NaN   
Anthropology                                        4.0  0.577350  0.577350   
Anthropology                                        3.0       NaN       NaN   
Architecture                                        6.0  1.732051  1.000000   
Architecture                                        5.0       NaN       NaN   
Art                                                 5.5  2.366432  0.816497   

                                                                 len          \
variable                                                calm anxious bookish   
major                                                                          
 None yet                                                NaN       1       1   
+ACI-+ACIAIg-hotel and restaurant management+AC...       NaN       1       1   
Aerospace Engineer                                       NaN       1       1   
Aerospace Engineering                                    NaN       1       1   
Agricultural Economics                                   NaN       1       1   
Anthropology                                        2.516611       3       3   
Anthropology                                             NaN       1       1   
Architecture                                        1.527525       3       3   
Architecture                                             NaN       1       1   
Art                                                 1.211060    

<a id='examining_multiindex'></a>

### The Inner Workings of the MultiIndex

--- 

The `.names` attribute on the index and columns will show you the hierarchy of labels. The row index is `'major'`, and the two column indices are `None` and `'variable'` (The aggregate functions get no label from `.pivot_table()` in this case). 

If you print out the columns, you can see the data set has become a `pandas` `MultiIndex` object that has levels, labels, and names.

In [49]:
print nerdy_major_summary.index.names

[u'major']


In [50]:
print nerdy_major_summary.columns.names

[None, u'variable']


In [51]:
print nerdy_major_summary.columns

MultiIndex(levels=[[u'mean', u'median', u'std', u'len'], [u'anxious', u'bookish', u'calm']],
           labels=[[0, 0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3], [0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2]],
           names=[None, u'variable'])


Indexing along the hierarchical column headers can be done with chained bracket keys — i.e., including the top-level column label in the first bracket, and so on, down to the bottom level.

In [28]:
nerdy_major_summary['mean'].head(2)

variable,anxious,bookish,calm
major,,,
None yet,7.0,3.0,3.0
+ACI-+ACIAIg-hotel and restaurant management+ACIAIg-+ACI-,2.0,2.0,7.0


In [29]:
nerdy_major_summary['mean']['anxious'].head(2)

major
 None yet                                                    7.0
+ACI-+ACIAIg-hotel and restaurant management+ACIAIg-+ACI-    2.0
Name: anxious, dtype: float64

In [30]:
nerdy_major_summary['mean'][['anxious','bookish']].head(2)

variable,anxious,bookish
major,,
None yet,7.0,3.0
+ACI-+ACIAIg-hotel and restaurant management+ACIAIg-+ACI-,2.0,2.0


In some cases, you can just split them up by comma within the brackets.

In [31]:
nerdy_major_summary['mean', 'bookish'].head(2)

major
 None yet                                                    3.0
+ACI-+ACIAIg-hotel and restaurant management+ACIAIg-+ACI-    2.0
Name: (mean, bookish), dtype: float64

<a id='multiindex_to_flat'></a>

### Getting Rid of the MultiIndex: "Flattening" Data

---

MultiIndex DataFrames hold great potential and are a cool concept. That being said, the overhead and confusion on how to subset/mask them is most often not worth it, especially when your data needs to be formatted for insertion into a model.

The most reliable way to "flatten" a MultiIndexed DataFrame is with the `.to_records()` function. To make this a new DataFrame, it needs to be wrapped in a `pd.DataFrame()` like so:

In [32]:
nerdy_major_flat = pd.DataFrame(nerdy_major_summary.to_records())
nerdy_major_flat.head(2)

,major,"('mean', 'anxious')","('mean', 'bookish')","('mean', 'calm')","('median', 'anxious')","('median', 'bookish')","('median', 'calm')","('std', 'anxious')","('std', 'bookish')","('std', 'calm')","('len', 'anxious')","('len', 'bookish')","('len', 'calm')"
0,None yet,7.0,3.0,3.0,7.0,3.0,3.0,NaN,NaN,NaN,1,1,1
1,+ACI-+ACIAIg-hotel and restaurant management+A...,2.0,2.0,7.0,2.0,2.0,7.0,NaN,NaN,NaN,1,1,1


You can see that the new column names are tuples of the hierarchy of MultiIndexed columns. For example, you could convert these to new, more easily indexed columns with something like a list comprehension.

The **`.eval()`** function takes a string and trys to evaluate it as if it were a Python command.

**Use a list comprehension and the `.eval()` function to convert the flattened MultiIndexed columns to something more readable.**

In [33]:
nerdy_major_flat.columns = ['_'.join(eval(c)) if c[0] == '(' else c for c in nerdy_major_flat.columns]
nerdy_major_flat.head(2)

,major,mean_anxious,mean_bookish,mean_calm,median_anxious,median_bookish,median_calm,std_anxious,std_bookish,std_calm,len_anxious,len_bookish,len_calm
0,None yet,7.0,3.0,3.0,7.0,3.0,3.0,NaN,NaN,NaN,1,1,1
1,+ACI-+ACIAIg-hotel and restaurant management+A...,2.0,2.0,7.0,2.0,2.0,7.0,NaN,NaN,NaN,1,1,1


<a id='merging_joining_preface'></a>

### A Preface: Merging and Joining With Long and Wide Format Data

---

You will be merging and joining data sets extensively throughout this course and in your future careers. However, it is important to note the differences between merging long and wide data sets together.

**Load in the data used above, but now split it so that the demographic variables are in one data set and the survey question answers are in another.** 

These data sets are in a wide format, and they both contain `subject_id`s to identify the questions' categories. 

As you may recall, the demographic responses have fewer observations.

In [34]:
n_demos_file = '../datasets/NPAS_parsed_trunc_demo_sample.csv'
n_survey_file = '../datasets/NPAS_parsed_trunc_survey.csv'

demos_subset = pd.read_csv(n_demos_file)
survey = pd.read_csv(n_survey_file)

In [35]:
print demos_subset.shape, survey.shape

(700, 12) (1391, 46)


In [36]:
demos_subset.head(2)

,education,urban,gender,engnat,age,hand,religion,voted,married,familysize,major,subject_id
0,4.0,2.0,2.0,1.0,50.0,1.0,1.0,1.0,1.0,3.0,biophysics,1
1,3.0,1.0,2.0,2.0,22.0,1.0,1.0,1.0,1.0,2.0,biology,2


In [37]:
survey.head(2)

,race_white,race_nerdy,race_native_american,writing_novel,read_tech_reports,online_over_inperson,introspective,hobbies_over_people,books_over_parties,bookish,...,reserved,conventional,was_odd_child,prefer_fictional_people,enjoy_learning,excited_about_research,strange_person,like_superheroes,socially_awkward,subject_id
0,1.0,0.0,0.0,3.0,5.0,4.0,5.0,4.0,5.0,5.0,...,7.0,1.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,0
1,1.0,0.0,0.0,1.0,4.0,3.0,3.0,1.0,4.0,4.0,...,5.0,1.0,3.0,3.0,3.0,4.0,4.0,4.0,5.0,1


In [38]:
print demos_subset.columns
print survey.columns

Index([u'education', u'urban', u'gender', u'engnat', u'age', u'hand',
       u'religion', u'voted', u'married', u'familysize', u'major',
       u'subject_id'],
      dtype='object')
Index([u'race_white', u'race_nerdy', u'race_native_american', u'writing_novel',
       u'read_tech_reports', u'online_over_inperson', u'introspective',
       u'hobbies_over_people', u'books_over_parties', u'bookish',
       u'libraries_over_publicspace', u'race_native_austrailian',
       u'like_hard_material', u'race_hispanic', u'diagnosed_autistic',
       u'play_many_videogames', u'race_arab', u'race_asian',
       u'interested_science', u'playes_rpgs', u'in_advanced_classes',
       u'collect_books', u'intelligence_over_appearance',
       u'watch_science_shows', u'academic_over_social',
       u'like_science_fiction', u'like_dry_topics', u'race_black', u'calm',
       u'disorganized', u'extraverted', u'dependable', u'critical',
       u'opennness', u'anxious', u'sympathetic', u'reserved', u'convention

<a id='pandas_merge'></a>

### `pandas`' `.merge()` function: Joining Long Format vs. Wide Format Data

---

The `.merge()` function comes built into a DataFrame. The first argument is another DataFrame you want to merge it with, and the `on` keyword argument is the key(s) by which you want the DataFrames to be "matched."

We are specifying `how='inner'` here, which means that the key must be present in both DataFrames to have the corresponding rows included in the output. Because the demographics data set has fewer `subject_id`s, it will only merge the `subject_id` rows from the survey data set that are also present in the demographics data set.

**Combine the survey and demographic wide format data sets using `.merge()`.**

In [39]:
demos_survey = demos_subset.merge(survey, on=['subject_id'], how='inner')

In [40]:
print demos_survey.shape
demos_survey.head(2)

(700, 57)


,education,urban,gender,engnat,age,hand,religion,voted,married,familysize,...,sympathetic,reserved,conventional,was_odd_child,prefer_fictional_people,enjoy_learning,excited_about_research,strange_person,like_superheroes,socially_awkward
0,4.0,2.0,2.0,1.0,50.0,1.0,1.0,1.0,1.0,3.0,...,5.0,5.0,1.0,3.0,3.0,3.0,4.0,4.0,4.0,5.0
1,3.0,1.0,2.0,2.0,22.0,1.0,1.0,1.0,1.0,2.0,...,2.0,7.0,1.0,5.0,5.0,5.0,5.0,5.0,3.0,5.0


**Convert the demographic and survey data into long format using `.melt()`.**

- For the demographic DataFrame, specify two `id_vars` — `gender` and `subject_id`.
- For the survey DataFrame, only specify `subject_id` for `id_vars`.

In [41]:
demos_long = pd.melt(demos_subset, id_vars=['subject_id','gender'])
print demos_long.shape
demos_long.head(2)

(7000, 4)


,subject_id,gender,variable,value
0,1,2.0,education,4
1,2,2.0,education,3


In [42]:
survey_long = pd.melt(survey, id_vars=['subject_id'])
print survey_long.shape
survey_long.head(2)

(62595, 3)


,subject_id,variable,value
0,0,race_white,1.0
1,1,race_white,1.0


**Merge the long form data sets together, just like we did previously with the wide format data.**

Here, we will still merge on `subject_id`, using `'inner'` for the `how` variable. We have duplicate named columns in each of these DataFrames (`variable` and `value`). We can specify `suffixes=('_survey','_demo')` to give the instances of the survey and demographic DataFrames appropriate column names when they are joined together.

In [43]:
demos_survey_long = survey_long.merge(demos_long, on='subject_id', how='inner',
                                      suffixes=('_survey','_demo'))
print demos_survey_long.shape
demos_survey_long.head(3)

(315000, 6)


,subject_id,variable_survey,value_survey,gender,variable_demo,value_demo
0,1,race_white,1.0,2.0,education,4
1,1,race_white,1.0,2.0,urban,2
2,1,race_white,1.0,2.0,engnat,1
